# Parquet konvertering-, partisjonering- og filtreringsverktøy for AIS-data

Dette verktøyet konverterer Parquet-filer til GeoParquet-format, partisjonerer på angitte kolonner og filtrerer på angitt dato og radius rundt Kristiansand sentrum. Data vises på tabell og kart.

Verktøyet er en løsningsmodell som bruker GeoPandas for følgende brukerhistorie:

"Som havnesjef i Kristiansand kommune ønsker jeg å vite hvilke skip som befant seg innen 6 km fra Kristiansand sentrum på en bestemt dato."

### Hvordan bruke dette verktøyet
1. Last opp dine geografiske datafiler til mappen `data/raw`
2. Kjør alle cellene i denne notebooken
3. De konverterte filene vil bli lagret i mappen `data/processed`

### Støttede filformater

- Parquet-filer med geografisk informasjon

## Last inn nødvendige pakker

Kjør cellen nedenfor for å installere og importere pakkene som verktøyet trenger:

In [ ]:
!pip install -r requirements.txt

In [ ]:
import os
import argparse
import pandas as pd
import geopandas as gpd
from shapely import wkt

from shapely import wkb
from shapely.geometry import Point, Polygon
import time
import shutil
from datetime import datetime
import os
import datetime

import glob
import random

import folium
from folium import plugins
from shapely.geometry import Point, Polygon

from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

from shapely.geometry import Point
import pyproj
from shapely.ops import transform
from functools import partial


### Globale variabler

In [ ]:
_current_observer = None
global_paths = None

# Sett standardverdier for parameterne
data_dir = "./data"

### Definer funksjoner
Kjør cellen nedenfor for å definere nødvendige funksjoner:

In [ ]:
# ---------- MAPPESTRUKTUR ---------- #

def create_folders(data_folder="./data"):
    """
    Oppretter nødvendig mappestruktur for konvertering.

    Args:
        data_folder: Sti til hovedmappen for data

    Returns:
        dict: Stier til opprettede mapper
    """
    # Definer mappestruktur
    raw_folder = os.path.join(data_folder, "raw")
    processed_folder = os.path.join(data_folder, "processed")
    source_folder = os.path.join(data_folder, "source")
    incoming_folder = os.path.join(data_folder, "incoming")

    # Opprett mapper
    os.makedirs(data_folder, exist_ok=True)
    os.makedirs(raw_folder, exist_ok=True)
    os.makedirs(processed_folder, exist_ok=True)
    os.makedirs(source_folder, exist_ok=True)
    os.makedirs(incoming_folder, exist_ok=True)

    # Returner stier for senere bruk
    return {
        "raw_folder": os.path.abspath(data_folder),
        "processed_folder": os.path.abspath(raw_folder),
        "prosessert_mappe": os.path.abspath(processed_folder),
        "source_folder": os.path.abspath(source_folder),
        "incoming_folder": os.path.abspath(incoming_folder)
    }


# ---------- KONVERTERING & PARTISJONERING ---------- #

def add_time_partitioning_columns(df, time_column='date_time_utc'):
    """
    Legger til kolonner for tidspartisjonering (kun time for partisjonering, men beholder dato-kolonner).
    """
    if time_column not in df.columns:
        print(f"Advarsel: Tidsstempelkolonne '{time_column}' finnes ikke")
        return df

    if not pd.api.types.is_datetime64_any_dtype(df[time_column]):
        print(f"Advarsel: Kolonnen '{time_column}' er ikke en datetime-kolonne")
        return df

    df_with_time = df.copy()
    df_with_time['year'] = df_with_time[time_column].dt.year
    df_with_time['month'] = df_with_time[time_column].dt.month
    df_with_time['day'] = df_with_time[time_column].dt.day
    df_with_time['hour'] = df_with_time[time_column].dt.hour

    return df_with_time

def create_geodataframe(df):
    """
    Oppretter en GeoDataFrame fra en DataFrame ved å finne koordinater eller geometrikolonner.
    """

    # Sjekk for lat/long kolonner
    if 'longitude' in df.columns and 'latitude' in df.columns:
        try:
            return gpd.GeoDataFrame(
                df,
                geometry=gpd.points_from_xy(df.longitude, df.latitude),
                crs="EPSG:4326"
            )
        except Exception as e:
            print(f"Kunne ikke opprette geometri fra lat/long: {e}")

    # Sjekk for andre geometrikolonner
    geom_columns = [col for col in df.columns if any(
        term in col.lower() for term in ['geom', 'coord', 'point', 'polygon', 'linestring', 'wkt']
    )]

    for col in geom_columns:
        if df[col].dtype != 'object':
            continue

        try:
            geom = df[col].apply(wkt.loads)
            return gpd.GeoDataFrame(df, geometry=geom, crs="EPSG:4326")
        except Exception:
            continue

    return None

def save_partitioned_geoparquet(gdf, output_path, partition_columns):
    """
    Lagrer en GeoDataFrame som partisjonert GeoParquet.
    """

    # Konverterer geometri til WKB for å kunne partisjonere
    df_med_wkb = gdf.copy()
    df_med_wkb['geometry_wkb'] = df_med_wkb['geometry'].apply(lambda geom: wkb.dumps(geom))
    df_for_partisjon = df_med_wkb.drop(columns=['geometry'])

    # Utfør partisjonering
    df_for_partisjon.to_parquet(output_path, partition_cols=partition_columns)

    # Konverter hver partisjonert fil tilbake til GeoParquet
    convert_partitioned_files_to_geoparquet(output_path)
    return output_path

def create_year_folder(year, output_path):
    """Oppretter en mappe for et spesifikt år."""
    year_folder = os.path.join(output_path, f"year={year}")
    os.makedirs(year_folder, exist_ok=True)
    return year_folder

def prepare_wkb_dataframe(year_df):
    """Forbereder en dataframe med WKB-konvertert geometri for partisjonering."""
    # Konverter geometri til WKB for å kunne partisjonere
    year_df_wkb = year_df.copy()
    year_df_wkb['geometry_wkb'] = year_df_wkb['geometry'].apply(lambda geom: wkb.dumps(geom))
    year_df_wkb = year_df_wkb.drop(columns=['geometry'])
    return year_df_wkb

def convert_file_to_geoparquet(src_file, dst_file):
    """Konverterer en enkelt fil fra WKB-format til GeoParquet."""
    try:
        part_df = pd.read_parquet(src_file)
        if 'geometry_wkb' not in part_df.columns:
            return False

        part_df['geometry'] = part_df['geometry_wkb'].apply(lambda x: wkb.loads(x))
        part_df = part_df.drop(columns=['geometry_wkb'])
        part_gdf = gpd.GeoDataFrame(part_df, geometry='geometry', crs="EPSG:4326")

        # Sørg for at målmappen eksisterer
        os.makedirs(os.path.dirname(dst_file), exist_ok=True)
        part_gdf.to_parquet(dst_file)
        return True
    except Exception as e:
        print(f"Advarsel: Kunne ikke konvertere {src_file} til GeoParquet: {e}")
        return False

def copy_and_convert_files(temp_year_folder, year_folder):
    """Kopierer og konverterer filer fra temp-mappen til målmappen."""
    success_count = 0
    error_count = 0

    # Opprett først alle mappene
    for root, dirs, _ in os.walk(temp_year_folder):
        for directory in dirs:
            src_dir = os.path.join(root, directory)
            rel_path = os.path.relpath(src_dir, temp_year_folder)
            dst_dir = os.path.join(year_folder, rel_path)
            os.makedirs(dst_dir, exist_ok=True)

    # Kopier og konverter filene
    for root, _, files in os.walk(temp_year_folder):
        for file in files:
            src_file = os.path.join(root, file)
            rel_path = os.path.relpath(src_file, temp_year_folder)
            dst_file = os.path.join(year_folder, rel_path)

            if convert_file_to_geoparquet(src_file, dst_file):
                success_count += 1
            else:
                error_count += 1

    return success_count, error_count

def convert_partitioned_files_to_geoparquet(root_folder):
    """
    Konverterer alle partisjonerte parquet-filer til GeoParquet format.
    """
    error_count = 0

    for root, _, files in os.walk(root_folder):
        for file in files:
            if not file.endswith('.parquet'):
                continue

            parquet_path = os.path.join(root, file)
            try:
                # Les dataframe
                part_df = pd.read_parquet(parquet_path)

                # Hopp over hvis den ikke har geometry_wkb
                if 'geometry_wkb' not in part_df.columns:
                    continue

                # Konverter WKB tilbake til geometri
                part_df['geometry'] = part_df['geometry_wkb'].apply(lambda x: wkb.loads(x))
                part_df = part_df.drop(columns=['geometry_wkb'])

                # Lag GeoDataFrame
                part_gdf = gpd.GeoDataFrame(part_df, geometry='geometry', crs="EPSG:4326")

                # Skriv GeoParquet-filen
                part_gdf.to_parquet(parquet_path)
            except Exception as e:
                print(f"Feil ved konvertering av {parquet_path}: {e}")
                error_count += 1

    if error_count > 0:
        print(f"Advarsel: {error_count} filer kunne ikke konverteres til GeoParquet")

def convert_parquet_to_geoparquet(file_path, output_path, partition_columns=None):
    """
    Konverterer parquet-fil til GeoParquet-format med partisjonering.
    Args:
        file_path: Sti til parquet-filen
        output_path: Sti hvor GeoParquet-filen skal lagres
        partition_columns: Liste av kolonnenavn som skal brukes for partisjonering
    """
    try:
        df = pd.read_parquet(file_path)
    except Exception as e:
        print(f"Kunne ikke lese parquet-fil: {e}")
        return False

    # Håndter tidspartisjonering
    time_partitioning = False
    if partition_columns and 'date_time_utc' in partition_columns:
        partition_columns.remove('date_time_utc')
        time_partitioning = True

    # Legg til alle tidspartisjoneringskolonner, men vi vil bare partisjonere på time
    if time_partitioning:
        df = add_time_partitioning_columns(df)
        # Sett opp partisjonering kun på 'hour'
        partition_columns = ['hour'] + (partition_columns or [])

    # Opprett GeoDataFrame
    gdf = create_geodataframe(df)
    if gdf is None:
        return False

    # Sjekk at alle partisjoneringskolonner finnes
    if partition_columns and not all(col in gdf.columns for col in partition_columns):
        print(f"Advarsel: Ikke alle partisjoneringskolonner finnes i datasettet")
        missing = [col for col in partition_columns if col not in gdf.columns]
        print(f"Manglende kolonner: {missing}")
        return False

    # Lagre med partisjonering (bare time)
    if partition_columns:
        return save_partitioned_geoparquet(gdf, output_path, partition_columns)
    else:
        gdf.to_parquet(output_path)
        return output_path

def convert_all_parquet_files(data_folder, partition_columns=None):
    """
    Konverterer alle parquet-filer i en mappe til GeoParquet.
    """
    raw_folder = os.path.join(data_folder, "raw")
    processed_folder = os.path.join(data_folder, "processed")

    # Opprett mapper
    os.makedirs(raw_folder, exist_ok=True)
    os.makedirs(processed_folder, exist_ok=True)

    results = {
        "converted": [],
        "failed": []
    }

    # Finn alle parquet-filer
    parquet_files = [f for f in os.listdir(raw_folder)
                     if f.lower().endswith('.parquet') and os.path.isfile(os.path.join(raw_folder, f))]

    if not parquet_files:
        print("Ingen parquet-filer funnet i råmappen")
        return results

    # Konverter hver fil
    for filename in parquet_files:
        file_path = os.path.join(raw_folder, filename)
        base_filename = os.path.splitext(filename)[0]
        output_path = os.path.join(processed_folder, f"{base_filename}.parquet")

        # Konverter filen
        resultat = convert_parquet_to_geoparquet(
            file_path,
            output_path,
            partition_columns.copy() if partition_columns else None,
        )

        if resultat:
            results["converted"].append(file_path)
        else:
            results["failed"].append(file_path)
            print(f"Kunne ikke konvertere: {file_path}")

    return results

def start_conversion(data_folder="./data", partition_columns=None):
    """
    Start konverteringsprosessen fra Parquet til GeoParquet.
    Args:
        data_folder: Sti til datamappen
        partition_columns: Liste av kolonner for partisjonering
    """
    start_time = time.time()
    print(f"Starter konvertering og partisjonering av parquet-filer i {data_folder} ...\n")
    print(f"Partisjonering vil skje på time og følgende kolonner: {partition_columns or []}")

    # Opprett og sjekk mappene
    folder_paths = create_folders(data_folder)
    raw_folder = folder_paths["raw_folder"]

    # Utfør konverteringen
    results = convert_all_parquet_files(data_folder, partition_columns)

    print("Konvertering og partisjonering utført:")
    print(f"• Total behandlingstid: {time.time() - start_time:.2f} sekunder")
    print(f"• Konverterte filer: {len(results['converted'])}")

    return results

# ---------- FILTRERING ---------- #

def define_agder_polygon():
    """
    Definerer et forenklet polygon for Agder-kysten.

    Returns:
        Shapely Polygon som definerer Agder-kystlinjen
    """

    agder_coords = [
        (6.8, 57.8),  # Sørvest
        (10.0, 57.8),  # Sørøst
        (10.0, 59.5),  # Nordøst
        (6.8, 59.5),  # Nordvest
        (6.8, 57.8)   # Lukk polygonet
    ]
    return Polygon(agder_coords)

def filter_ais_data(
    root_folder,
    date=None,  # datetime objekt for dato - påkrevd
    filters=None,  # Dict med kolonner og verdier å filtrere på
    agder_polygon=None,  # GeoDataFrame eller Shapely polygon for Agder-kysten
    max_number_of_ships=10  # Maksimalt antall unike skip å returnere
):
    """
    Filtrerer AIS-data basert på dato og geografisk område.

    Args:
        root_folder: Mappen hvor dataene er lagret
        date: Dato for filtrering
        filters: Dictionary med kolonnenavn og verdier for filtrering
        agder_polygon: Polygon som definerer Agder-kysten
        max_number_of_ships: Maksimalt antall skip å returnere

    Returns:
        GeoDataFrame med filtrerte skip
    """
    if not date:
        print("Dato er påkrevd")
        return None

    if filters is None:
        filters = {}

    # Dato-komponenter for filtrering senere
    year, month, day = date.year, date.month, date.day

    # Konverter dato til string for bruk i filnavn
    dato_str = date.strftime('%Y-%m-%d')


    # Sjekk om rot-mappen eksisterer
    if not os.path.exists(root_folder):
        print(f"Mappen {root_folder} finnes ikke")
        return None

    # Finn alle mapper i rot-mappen
    date_folders = []
    for element in os.listdir(root_folder):
        full_path = os.path.join(root_folder, element)
        if os.path.isdir(full_path):
            date_folders.append(element)

    if not date_folders:
        print(f"Ingen datamapper funnet i {root_folder}")
        return None

    # Finn alle parquet-filer rekursivt
    date_found = []

    for date_folder in date_folders:
        base_path = os.path.join(root_folder, date_folder)
        # Traverser mappestrukturen rekursivt
        for root, dirs, files in os.walk(base_path):
            # For time-partisjonering, sjekk fil-innhold istedenfor å basere på katalogstruktur
            for file in files:
                if file.endswith('.parquet'):
                    file_path = os.path.join(root, file)
                    date_found.append(file_path)

    if not date_found:
        print(f"Ingen parquet-filer funnet")
        return None

    # Les inn og filtrer dataene
    dfs = []
    for file_path in date_found:
        try:
            # Ekstraher partisjonsverdier fra stien (for time)
            time_value = None
            parts = file_path.split(os.sep)
            for part in parts:
                if part.startswith('hour='):
                    time_value = int(part.split('=')[1])

            # Les parquet-filen
            df = pd.read_parquet(file_path)

            # Legg til hour som kolonne hvis den ikke allerede finnes
            if time_value is not None and 'hour' not in df.columns:
                df['hour'] = time_value

            # Filtrer på dato
            if 'year' in df.columns and 'month' in df.columns and 'day' in df.columns:
                df = df[(df['year'] == year) & (df['month'] == month) & (df['day'] == day)]

                if df.empty:
                    # Hopp over denne filen hvis ingen rader matcher datoen
                    continue

            # Utfør ekstra filtrering basert på filtre-parameteren
            ship_df = False
            for columns, value in filters.items():
                if columns not in df.columns:
                    ship_df = True
                    break

                # For alle kolonner, filtrer på vanlig måte
                if isinstance(value, list):
                    if not df[columns].isin(value).any():
                        ship_df = True
                        break
                else:
                    if not (df[columns] == value).any():
                        ship_df = True
                        break

            if not ship_df:
                # Gjør ytterligere filtrering på dataframe-nivå
                for columns, value in filters.items():
                    if columns in df.columns:
                        if isinstance(value, list):
                            df = df[df[columns].isin(value)]
                        else:
                            df = df[df[columns] == value]

                if not df.empty:
                    dfs.append(df)

        except Exception as e:
            print(f"Feil ved lesing av {file_path}: {str(e)}")

    if not dfs:
        print("Ingen data lest inn")
        return None

    # Slå sammen alle dataframes
    combined_df = pd.concat(dfs, ignore_index=True)

    # Konverter til GeoDataFrame hvis koordinater finnes
    if 'longitude' in combined_df.columns and 'latitude' in combined_df.columns:
        # Opprett geometrikolonne fra longitude og latitude
        geometry = [Point(xy) for xy in zip(combined_df['longitude'], combined_df['latitude'])]
        geo_df = gpd.GeoDataFrame(combined_df, geometry=geometry, crs="EPSG:4326")

        # Filtrer basert på Agder-polygon hvis gitt
        if agder_polygon is not None:
            # Sjekk om agder_polygon er et GeoDataFrame eller et Shapely-objekt
            if isinstance(agder_polygon, gpd.GeoDataFrame):
                # Sørg for at CRS er likt
                if agder_polygon.crs != geo_df.crs:
                    agder_polygon = agder_polygon.to_crs(geo_df.crs)

                # Spatial join - dette kan ta litt tid for store datasett
                geo_df = gpd.sjoin(geo_df, agder_polygon, how="inner", predicate="within")
            else:
                # Antar at agder_polygon er et Shapely-objekt
                geo_df = geo_df[geo_df.geometry.within(agder_polygon)]

        # Sorter etter nyeste data først for hvert skip
        if 'timestamp' in geo_df.columns:
            geo_df = geo_df.sort_values(by='timestamp', ascending=False)

        # Begrens til unike skip
        if 'mmsi' in geo_df.columns or 'ship_name' in geo_df.columns:
            id_columns = 'mmsi' if 'mmsi' in geo_df.columns else 'ship_name'
            unique_ships = geo_df[id_columns].unique()

            if len(unique_ships) > max_number_of_ships:
                print(f"Begrenser til {max_number_of_ships} skip")
                unique_ships = unique_ships[:max_number_of_ships]

            geo_df = geo_df[geo_df[id_columns].isin(unique_ships)]

            # Hent første rad for hvert unikt skip for å få en kompakt liste
            geo_df = geo_df.drop_duplicates(subset=[id_columns])

        return geo_df
    else:
        print("Mangler koordinater (longitude/latitude)")
        return None

def find_ships_near_point(ship_gdf, point, buffer_distance=500):
    """
    Filtrerer skip som er innenfor en gitt avstand fra et punkt.

    Args:
        ship_gdf: GeoDataFrame med skip
        point: Tuple (longitude, latitude)
        buffer_distance: Avstand i meter

    Returns:
        GeoDataFrame med filtrerte skip
    """
    if ship_gdf is None or ship_gdf.empty:
        return None

    try:

        # Opprett punkt fra koordinater
        center_point = Point(point)

        # Projiser til et lokalt koordinatsystem som bruker meter som enheter (UTM-sone)
        # Finn UTM-sone basert på longitude
        utm_sone = int((point[0] + 180) / 6) + 1
        utm_crs = f"EPSG:326{utm_sone}"  # Nordlig halvkule

        # Transformer funksjonen for å lage buffer i meter
        project_to_utm = pyproj.Transformer.from_crs(
            "EPSG:4326", utm_crs, always_xy=True).transform
        project_to_wgs84 = pyproj.Transformer.from_crs(
            utm_crs, "EPSG:4326", always_xy=True).transform

        # Projiser punkt til UTM, lag buffer i meter, projiser tilbake til WGS84
        utm_point = transform(project_to_utm, center_point)
        buffer_utm = utm_point.buffer(buffer_distance)
        buffer_wgs84 = transform(project_to_wgs84, buffer_utm)

        # Filtrer for skip innenfor bufferen
        ships_within_buffer = ship_gdf[ship_gdf.geometry.within(buffer_wgs84)]
        print(f"Fant {len(ships_within_buffer)} skip innenfor {buffer_distance}m fra punktet")

        return ships_within_buffer, buffer_wgs84

    except Exception as e:
        print(f"Feil ved buffering: {str(e)}")
        return None, None

def filtrer_skip_agder(root_folder, dato, radius=None, center_point=(8.0182, 58.1599), number_of_ships=10):
    """
    Filtrerer skip i Agder-området, enten basert på et definert polygon eller en radius fra et senterpunkt.

    Args:
        root_folder: Mappen hvor de konverterte dataene er lagret
        dato: Datoen å filtrere på (datetime objekt)
        radius: Radius i meter fra senterpunkt (None = bruk polygon)
        center_point: Tuple (longitude, latitude) for senterpunkt (standard: Kristiansand)
        number_of_ships: Maksimalt antall skip å returnere

    Returns:
        GeoDataFrame med filtrerte skip
    """
    # Først hent alle skip for datoen uten geografisk filtrering
    ships_on_date = filter_ais_data(
        root_folder=root_folder,
        date=dato,
        filters=None,
        agder_polygon=None,  # Ingen geografisk filter her ennå
        max_number_of_ships=100  # Hent litt flere for å kunne filtrere geografisk etterpå
    )

    if ships_on_date is None or ships_on_date.empty:
        print(f"Ingen skip funnet på datoen {dato.strftime('%d.%m.%Y')}")
        return None

    # Hvis radius er oppgitt, bruk finn_skip_rundt_punkt
    if radius is not None and radius > 0:
        print(f"Søker etter skip innen {radius} meter fra senterpunkt")
        ships_near_point, buffer = find_ships_near_point(
            ships_on_date,
            center_point,
            buffer_distance=radius
        )

        # Begrens antall skip hvis nødvendig
        if ships_near_point is not None and len(ships_near_point) > number_of_ships:
            ships_near_point = ships_near_point.head(number_of_ships)

        if ships_near_point is None or ships_near_point.empty:
            print(f"Ingen skip funnet innenfor {radius}m fra senterpunkt på datoen {dato.strftime('%d.%m.%Y')}")
            return None

        print(f"Fant {len(ships_near_point)} skip innenfor {radius}m på datoen {dato.strftime('%d.%m.%Y')}")
        return ships_near_point
    else:
        # Bruk standard Agder-polygon
        agder_polygon = define_agder_polygon()

        # Filtrer med polygon
        ships_within_agder = ships_on_date[ships_on_date.geometry.within(agder_polygon)]

        # Begrens antall skip
        if len(ships_within_agder) > number_of_ships:
            ships_within_agder = ships_within_agder.head(number_of_ships)

        if ships_within_agder.empty:
            print(f"Ingen skip funnet innenfor Agder-område på datoen {dato.strftime('%d.%m.%Y')}")
            return None

        print(f"Fant {len(ships_within_agder)} skip innenfor Agder-område på datoen {dato.strftime('%d.%m.%Y')}")
        return ships_within_agder


## Analyse over skip i Agder

In [ ]:
# Filtrer skip i Agder for en bestemt dato
root_folder = "data/processed"
date = datetime.datetime(2024, 5, 16)  # Velg dato
ships_in_agder = filtrer_skip_agder(root_folder, date)

# Vis skipene på tabellform
if ships_in_agder is not None and not ships_in_agder.empty:
    # Velg kolonner å vise
    columns = ['mmsi', 'ship_name', 'ship_type', 'longitude', 'latitude']

    # Velg bare kolonner som faktisk finnes i datasettet
    available_columns = [kol for kol in columns if kol in ships_in_agder.columns]

    styled_tabell = ships_in_agder[available_columns].style \
        .format({'longitude': '{:.4f}', 'latitude': '{:.4f}'}) \
        .set_caption('Skip innenfor Agder-området') \
        .set_table_styles([
        {'selector': 'th', 'props': [('background-color', '#f2f2f2'),
                                     ('color', '#333'),
                                     ('font-weight', 'bold')]},
        {'selector': 'td', 'props': [('padding', '5px')]}
    ])

    display(styled_tabell)
else:
    print("Ingen skip å vise.")